<a href="https://colab.research.google.com/github/Rishabh672003/Learning-machine-learning/blob/main/jupyter-notebooks/url_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# get dataset from kaggle

In [ ]:
### This requires an API token .json file from kaggle

### get it from going to https://www.kaggle.com/settings/account and clicking `create new token`

### Then place the token .json file in your google drive, and copy the location in `kaggle_creds_path` variable and command below it

from google.colab import drive
drive.mount("/content/drive")

kaggle_creds_path = "/content/drive/MyDrive/Kaggle/kaggle.json"
! cp /content/drive/MyDrive/Kaggle/kaggle.json .

! pip install kaggle --quiet

!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d shivamb/spam-url-prediction

import zipfile
import os
import shutil
location = '/content/spam-url-prediction'
zip_ref = zipfile.ZipFile(location+'.zip', 'r')

if os.path.isdir(location):
    shutil.rmtree(location)
    os.mkdir(location)
else:
    os.mkdir(location)

zip_ref.extractall(location)
zip_ref.close()

# Unmount your Google Drive
drive.flush_and_unmount()

Mounted at /content/drive
rm: cannot remove '/root/.kaggle': No such file or directory
  0% 0.00/3.41M [00:00<?, ?B/s]
100% 3.41M/3.41M [00:00<00:00, 138MB/s]


In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv("url_spam_classification.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148303 entries, 0 to 148302
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   url      148303 non-null  object
 1   is_spam  148303 non-null  bool  
dtypes: bool(1), object(1)
memory usage: 1.3+ MB


In [ ]:
df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [ ]:
label_encoder = LabelEncoder()
df['is_spam'] = label_encoder.fit_transform(df['is_spam'])

In [ ]:
df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,1
1,https://www.hvper.com/,1
2,https://briefingday.com/m/v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,1


In [ ]:
# Vectorization step
vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, max_df=0.8)

In [ ]:
# Create a pipeline that includes vectorization and scaling
preprocessor = Pipeline([
    ('tfidf', vectorizer),
    ('scaling', StandardScaler(with_mean=False))
])

In [40]:
X_train, X_test, y_train, y_test = train_test_split(df['url'], df['is_spam'], test_size=0.2,random_state=42)

In [41]:
# Preprocess the data
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [42]:
# Create the BaggingClassifier
model = BaggingClassifier()

In [43]:
# Train the model
model.fit(X_train_processed, y_train)

BaggingClassifier()

In [44]:
# Evaluate the model
score = model.score(X_test_processed, y_test)

In [45]:
print(f"Model accuracy: {score*100}")

Model accuracy: 92.14456693975254


In [ ]:
# Preprocess the new data
new_data_processed = preprocessor.transform(['https://www.kaggle.com/code/harshal0412/fake-url-prediction-using-ml-with-visualization'])

In [ ]:
# Make predictions on the new data
predictions = model.predict(new_data_processed)

# The predictions variable now contains the predicted labels for the new data
print(predictions)

[0]
